In [31]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
from scipy.stats import t

from logistic_growth import logistic_growth

In [18]:
data = pd.read_csv('../data/region/就业总人数.csv', index_col=0)

In [28]:
years = np.array([int(year) for year in data.columns])
time_points = years - min(years)

In [32]:
predictions = {}
confidence_intervals = {}

In [34]:
for region in data.index:
    employment = data.loc[region].dropna().values
    t_region = time_points[:len(employment)]

    # 初始参数猜测
    initial_guess = [max(employment), 0.1, employment[0]]

    # 拟合逻辑斯蒂模型
    try:
        params, covariance = curve_fit(logistic_growth, t_region, employment, p0=initial_guess)
        K, r, x0 = params
        
        # 计算参数标准差
        p_stderr = np.sqrt(np.diag(covariance))
        n = len(employment)  # 样本大小

        # t分布的临界值，对于95%置信区间
        t_critical = t.ppf(1 - 0.05 / 2, df=n - 1)

        # 计算每个参数的95%置信区间
        ci = t_critical * p_stderr

        # 预测2023年的就业人数
        t_future = 2023 - min(years)
        employment_pred = logistic_growth(t_future, K, r, x0)

        # 存储结果
        predictions[region] = employment_pred
        confidence_intervals[region] = ci

    except RuntimeError as e:
        print(f"未能拟合模型于地区：{region}。原因：{e}")

In [39]:
print("2023年各地区的就业人数预测及其95%置信区间：")
for region in predictions:
    K_pred = predictions[region]
    K_lower_bound = K_pred - confidence_intervals[region][0]
    K_upper_bound = K_pred + confidence_intervals[region][0]
    print(f"{region}: {K_pred:.2f} (95% CI for K: {K_lower_bound:.2f} to {K_upper_bound:.2f}) {K_lower_bound<=K_pred<=K_upper_bound}")

2023年各地区的就业人数预测及其95%置信区间：
北京: 1197.38 (95% CI for K: 1156.89 to 1237.86) True
天津: 688.68 (95% CI for K: 646.40 to 730.97) True
河北: 3846.64 (95% CI for K: 3715.69 to 3977.58) True
山西: 1811.15 (95% CI for K: 1703.26 to 1919.03) True
内蒙古: 1320.59 (95% CI for K: 1133.36 to 1507.82) True
辽宁: 2321.32 (95% CI for K: 2208.67 to 2433.97) True
吉林: 1356.66 (95% CI for K: 1284.54 to 1428.79) True
黑龙江: 1767.51 (95% CI for K: 1648.00 to 1887.02) True
上海: 1446.22 (95% CI for K: 1211.10 to 1681.33) True
江苏: 4901.05 (95% CI for K: 4819.75 to 4982.35) True
浙江: 3909.28 (95% CI for K: 3235.53 to 4583.02) True
安徽: 4022.87 (95% CI for K: 3724.71 to 4321.03) True
福建: 2258.06 (95% CI for K: 2165.59 to 2350.52) True
江西: 2332.06 (95% CI for K: 2287.42 to 2376.70) True
山东: 5719.26 (95% CI for K: 5633.23 to 5805.29) True
河南: 4869.60 (95% CI for K: 1056.16 to 8683.03) True
湖北: 3330.75 (95% CI for K: -282.80 to 6944.29) True
湖南: 3790.89 (95% CI for K: 3656.14 to 3925.63) True
广东: 7193.35 (95% CI for K: 6901.13 to 7

In [40]:
res = []

In [41]:
for region in predictions:
    K_pred = predictions[region]
    res.append([region, K_pred])

In [42]:
res_pd = pd.DataFrame(res, columns=['地区', '总人数'])

In [43]:
res_pd.to_csv("../data/region/就业总人数预测2023.csv", index=False)